In [1]:
import glob, json
import pandas as pd
from datetime import datetime, date
import shapely, fiona
from fiona import transform
import geopandas as gpd

In [2]:
def create_geodataframe(term='HeatPerimeter', fill_dates_forward=False):
    days = []
    for shp in [x for x in glob.glob('./data/**/*') if term in x and x.endswith('.shp')]:
        d  = shp.split("/")[3]
        file_date = datetime(int(d[:4]),
                             int(d[4:6]),
                             int(d[6:8]),
                             int(d[9:11]),
                             int(d[11:13]))
        raw  = fiona.open(shp)
        raw_geom = next(iter(raw))

        transformed_geom = transform.transform_geom(raw.crs['init'], 'EPSG:4326', raw_geom['geometry'])

        raw_geom['properties']['datetime'] = file_date
        raw_geom['properties']['timestamp'] = int(file_date.timestamp())
        raw_geom['properties']['day'] = file_date.date()
        raw_geom['properties']['geometry'] = shapely.geometry.shape(transformed_geom)

        days.append(raw_geom['properties'])
        df = pd.DataFrame(days).sort_values(by='timestamp').reset_index()
        
        if fill_dates_forward:
            df.drop_duplicates(subset='day', keep='last').set_index('day')
            r = pd.date_range(start=df.index.min(), end=df.index.max())
            df = df.reindex(r).fillna(method='ffill')
            df['day_ts'] = [int(d.timestamp()) for d in df.index]
    return gpd.GeoDataFrame(df).set_crs(epsg=4326)

In [3]:
gdf = create_geodataframe(term='HeatPerimeter', fill_dates_forward=False)
_slice = gdf[gdf.timestamp!=1629172680]
with open('docs/heat_perimeter.geojson','w') as outFile:
    outFile.write( _slice[['geometry', 'timestamp']].to_json() )
with open('docs/timestamps.jsonp','w') as outFile:
    outFile.write("const TIMESTAMPS = {}".format(sorted(list(_slice.timestamp))))

In [4]:
len(gdf), len(_slice)

(26, 25)

In [219]:
isolated_hot_spots = create_geodataframe(term='Isolated')
with open('hot_spots.geojson','w') as outFile:
    outFile.write( isolated_hot_spots[['geometry', 'timestamp', 'day_ts']].to_json() )